In [13]:
import telebot
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager

with open("telebotkey.txt", "r") as file:
    key = file.read().strip()

bot = telebot.TeleBot(key)

In [14]:
@bot.message_handler(commands=["start"])

def enviar(message):
    bot.reply_to(message, "Hola, Soy WalaYera bot y con mi ayuda te convertirás en el Rey de Wallapop")

In [15]:

@bot.message_handler(commands=["busca"])

def busca(message):
    
    chatid = message.chat.id

    bot.send_message(chat_id = chatid, text = f"Buenas, soy el bot WalaYera y te buscaré las tarjetas gráficas que quieras")
    URL = "https://es.wallapop.com/app/search?filters_source=quick_filters&keywords=targeta%20grafica&order_by=newest&"


    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(URL) 

    sleep(2)

    accept_terms_button = driver.find_element(By.XPATH, "/html/body/div/div[2]/div/div/div[2]/div/div/button[2]")
    if(accept_terms_button is not None):
        accept_terms_button.send_keys(Keys.RETURN)

    # TODO: skip tutorial
    sleep(25)
        
    # Crear una lista vacía para guardar las cards
    cardlist = []

    first = True

    while True:
        newcardlist = []
        # Encontrar todas las cards en la página
        cards = driver.find_elements(By.CLASS_NAME, "ItemCardList__item")

        # Iterar sobre las cards de forma inversa (de más reciente a más antigua)
        for card in cards[0:20]:
            # Obtener la información de la card
            # Obtener la imagen
            image = card.find_element(By.TAG_NAME, "img").get_attribute("src")
            
            # Obtener el título
            title = card.find_element(By.CLASS_NAME, "ItemCard__title").text
            
            # Obtener el precio
            price = card.find_element(By.CLASS_NAME, "ItemCard__price").text

            # Crear un diccionario con la información de la card
            card_info = {"image": image, "title": title, "price": price, "link":""}

            if not first:
                # Comprobar si la card ya está en la lista
                title_in_list = False
                for card2 in cardlist:
                    if card2 == title:
                        title_in_list = True
                        break

                if not title_in_list:
                    # Hacer clic en la card para acceder a su página
                    card.click()
                    # Esperar a que se cargue la página de la card
                    # Cambiar el enfoque del driver al nuevo tab
                    driver.switch_to.window(driver.window_handles[-1])
                    # Imprimir la URL obtenida
                    card_info['link'] = driver.current_url
                    # Cerrar el tab actual
                    driver.close()
                    # Cambiar el enfoque del driver al tab anterior
                    driver.switch_to.window(driver.window_handles[0])
                    
                    driver.execute_script("window.scrollTo(0, -1 * document.body.scrollHeight / 2);")

                    # Si la card no está en la lista, añadirla al principio
                    cardlist.insert(0, card_info['title'])

                    newcardlist.insert(0, card_info)
     
                else:
                    break
            else:
                 # Hacer clic en la card para acceder a su página
                card.click()
                # Esperar a que se cargue la página de la card
                # Cambiar el enfoque del driver al nuevo tab
                driver.switch_to.window(driver.window_handles[-1])
                # Imprimir la URL obtenida
                card_info['link'] = driver.current_url
                # Cerrar el tab actual
                driver.close()
                # Cambiar el enfoque del driver al tab anterior
                driver.switch_to.window(driver.window_handles[0])
                   
                driver.execute_script("window.scrollTo(0, -1 * document.body.scrollHeight / 2);")

                # Si la card no está en la lista, añadirla al principio
                cardlist.insert(-1, card_info['title'])
                newcardlist.insert(0, card_info)
                first = False

        for card in reversed(newcardlist):
            bot.send_message(chat_id=chatid, text=f"🖼️ Imagen: {card['image']}\n📝 Título: {card['title']} \n💰 Precio: {card['price']} \n 🔗 Link: {card['link']}" )    
            sleep(0.25) 

        sleep(30)
        driver.get(URL)
        sleep(2)

In [16]:
bot.polling()

C:\Users\Gerard\AppData\Local\Temp\ipykernel_24916\1818657838.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
